In [13]:
import numpy as np
import matplotlib.pyplot as plt
from graph_tools import Graph
import networkx as nx
import random
import copy
import time
import json

import ldpc
import bposd

from bposd.hgp import hgp
import pickle

import multiprocessing as mp
import random
from bposd.css import css_code
from ldpc import bp_decoder
from bposd import bposd_decoder
from ldpc.codes import ring_code

from scipy.optimize import curve_fit
import stim

import sys
sys.path.append("./src/")
from ErrorPlugin import *

from Decoders_SpaceTime import *
from Simulators_SpaceTime import *

# plt.style.use('/Users/qian/Documents/Python/PaperDoubleFig.mplstyle')

In [36]:
surface_d3 = hgp(ring_code(3), ring_code(3))

In [48]:
eval_code = surface_d3 # any quantum code using the qcode object
eval_p = 1e-3

p = eval_p
# circuit error params 
circuit_error_params = {"p_i": 0, "p_state_p": 0, "p_m": 0, "p_CX": 1, "p_idling_gate": 0}
# error params = circuit_error_params*p
error_params = {"p_i": circuit_error_params["p_i"]*p, "p_state_p": circuit_error_params["p_state_p"]*p, "p_m": circuit_error_params["p_m"]*p, "p_CX":circuit_error_params["p_CX"]*p, 
                "p_idling_gate": circuit_error_params["p_idling_gate"]*p}

num_rep = 3
# the total number of cycles, important: always set this one to be 3*N + 1 (N is the number of rounds), 
# in addition, it only takes odd values
num_cycles = 13 
eval_logical_type = 'Z' # only evaluating the logical Z failures
circuit_type = 'coloration' 

# generate the circuit fault graph using stim
circuit_simulator = CodeSimulator_Circuit_SpaceTime(code=eval_code, decoder1_z=None, decoder1_x=None, 
                        decoder2_z=None, decoder2_x=None, 
                        p=p, num_cycles=num_cycles, num_rep=num_rep, 
                        error_params=error_params, eval_logical_type=eval_logical_type, 
                        circuit_type = circuit_type, rand_scheduling_seed = 1)

circuit_simulator._generate_circuit() # generate the circuit and fault circuit
circuit_simulator._generate_circuit_graph() # generate the spacetime code, priors, etc.
circuit_fault_graphs = circuit_simulator.circuit_graph

# set up the decoder
max_iter = int(eval_code.N/10)
bp_method="minimum_sum"
ms_scaling_factor=0.625
osd_method="osd_e"
osd_order = 10

# BP spacetime decoder
dec1_z = ST_BP_Decoder_Circuit(h=circuit_fault_graphs['h1'], channel_probs=circuit_fault_graphs['channel_ps1'], 
                     max_iter=max_iter, bp_method=bp_method, ms_scaling_factor=ms_scaling_factor)

# the BPOSD decoder for the final transversal round
dec2_z = ST_BPOSD_Decoder_Circuit(h=circuit_fault_graphs['h2'], channel_probs=circuit_fault_graphs['channel_ps2'], 
                     max_iter=max_iter, bp_method=bp_method, ms_scaling_factor=ms_scaling_factor, 
                                 osd_method=osd_method, osd_order=osd_order)

circuit_simulator.decoder1_z = dec1_z 
circuit_simulator.decoder2_z = dec2_z

# eval_wer_per_code_list.append(circuit_simulator.WordErrorRate(num_samples=num_samples)[0])

In [47]:
# run the simulation and evaluate the WERs
num_samples = 10000
circuit_simulator.WordErrorRate(num_samples=num_samples)[0]

0.00019299501269032238

In [27]:
# generate the space-time tanner graph out of a stim circuit for BP
fault_circuit = circuit_simulator.fault_circuit
num_rounds = 4
num_rep = 3
num_logicals = eval_code.lx.shape[0]
H_list, L_list, channel_prob_list = GenFaultHyperGraph(str(fault_circuit.detector_error_model(flatten_loops=True)), num_rounds=num_rounds,
                                                               num_rep=num_rep, num_logicals=num_logicals)